### Retention Rate & Purchase Rate Analysis

In [5]:
import pandas as pd

file_path = '/Users/rachelyou/Downloads/dataset_v1.csv'
data= pd.read_csv(file_path)

data.head()

,Unnamed: 0,C1AccountNo,CXRecords,Comp Terr,1,2,3,4,5,6,7,8,9,10,11,12,Year,total_sales,encode
0,0,"A2061733054*,CNR",91470CV(.{82 W<,MCOW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,0.0,0
1,1,A2061733055$SC)`,91470EM$BC>5 W<,Southern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,0.0,0
2,2,A2061733055&RY+5,91470ER(#7ZS W<,Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,0.0,0
3,3,A2061733055(MK58,91470DV#294$ W<,Midwest,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,0.0,0
4,4,A2061733055)1@^M,91470EF%GL_Y W<,Central,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018,0.0,0


In [14]:
unique_customers_per_year = data.groupby('Year')['C1AccountNo'].nunique().reset_index()
unique_customers_per_year.columns = ['Year', 'UniqueCustomers']

data_sorted = data.sort_values(by=['C1AccountNo', 'Year'])

data_sorted['PreviousYear'] = data_sorted.groupby('C1AccountNo')['Year'].shift(1)

# Customers retained if their PreviousYear exists
data_sorted['Retained'] = data_sorted['PreviousYear'].notnull()

# Calculate retention rate for each year
retention_per_year = data_sorted.groupby('Year')['Retained'].mean().reset_index()
retention_per_year.columns = ['Year', 'RetentionRate']

# Convert the rate to percentages
retention_per_year['RetentionRate'] = retention_per_year['RetentionRate'] * 100

retention_per_year


,Year,RetentionRate
0,2018,0.0
1,2019,100.0
2,2020,100.0
3,2021,100.0
4,2022,100.0
5,2023,100.0
6,2024,100.0


In [15]:

# Filter customers who made purchases (total_sales > 0)
customers_with_purchases = data[data['total_sales'] > 0].copy()

# Sort the customers and calculate returning status (appeared in more than one year)
customers_with_purchases_sorted = customers_with_purchases.sort_values(by=['C1AccountNo', 'Year'])
customers_with_purchases_sorted['PreviousYear'] = customers_with_purchases_sorted.groupby('C1AccountNo')['Year'].shift(1)
customers_with_purchases_sorted['Retained'] = customers_with_purchases_sorted['PreviousYear'].notnull()

# Calculate the number of returning customers who made purchases each year
returning_customers_with_purchases_per_year = customers_with_purchases_sorted[customers_with_purchases_sorted['Retained']].groupby('Year')['C1AccountNo'].nunique().reset_index()
returning_customers_with_purchases_per_year.columns = ['Year', 'ReturningCustomersWithPurchases']

# Sort the entire dataset by year and calculate retention for all customers
data_sorted = data.sort_values(by=['C1AccountNo', 'Year'])
data_sorted['PreviousYear'] = data_sorted.groupby('C1AccountNo')['Year'].shift(1)
data_sorted['Retained'] = data_sorted['PreviousYear'].notnull()

# Calculate the total retention rate per year (including those who did not make purchases)
total_retention_per_year = data_sorted[data_sorted['Retained']].groupby('Year')['C1AccountNo'].nunique().reset_index()
total_retention_per_year.columns = ['Year', 'TotalReturningCustomers']

# Merge both results to compare returning customers vs those who made purchases
combined_retention_data = pd.merge(total_retention_per_year, returning_customers_with_purchases_per_year, on='Year', how='left')
combined_retention_data['PurchaseRate'] = (combined_retention_data['ReturningCustomersWithPurchases'] / combined_retention_data['TotalReturningCustomers']) * 100

combined_retention_data


,Year,TotalReturningCustomers,ReturningCustomersWithPurchases,PurchaseRate
0,2019,121602,8477,6.971102
1,2020,121602,7598,6.248252
2,2021,121602,2136,1.756550
3,2022,121602,2537,2.086314
4,2023,121602,9130,7.508100
5,2024,121602,5582,4.590385


From 2019 to 2024, the number of returning customers remained constant at 121,602, but the purchase rates fluctuated significantly. In 2019, 8,477 customers (6.97%) made purchases, dropping to 7,598 (6.25%) in 2020 and a sharp decline to 2,136 (1.76%) in 2021. A modest recovery occurred in 2022 with 2,537 customers (2.09%), followed by a peak in 2023 with 9,130 customers (7.51%). However, this trend did not hold in 2024, where the number fell to 5,582 (4.59%). These fluctuations suggest the need for consistent customer engagement strategies to sustain purchasing behavior.